In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxic-comment-classification-datasets/test2.csv
/kaggle/input/toxic-comment-classification-datasets/train.csv


In [2]:
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
pip install mtranslate

  Preparing metadata (setup.py) ... done
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3671 sha256=578040c1ca6536c478f85594d5ca56f0593f4183c1c34f38744d0268b39534de
  Stored in directory: /root/.cache/pip/wheels/c2/04/15/d7654c2c4a9a52e09922967593f3278fed66059be65ca671ea
Successfully built mtranslate
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install googletrans

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15721 sha256=e8bb38a42242dae9bde91979bcc354f22f3d3732c58f90b48b4d4b26590b53b7
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h1

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import pandas as pd

traindata = pd.read_csv('/kaggle/input/toxic-comment-classification-datasets/train.csv', encoding='ISO-8859-1')
print(traindata.head())


                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [7]:
testdata=pd.read_csv('/kaggle/input/toxic-comment-classification-datasets/test2.csv', encoding='ISO-8859-1')



In [8]:
len(traindata)

159571

In [9]:
len(testdata)

200

In [10]:
train=traindata[:round(0.7*len(traindata))]
#train_data=traindata['comment_text']
test=traindata[round(0.7*len(traindata)):]

In [11]:
from mtranslate import translate

def translate_to_english(text):
    try:
        translated_text = translate(text, 'en', 'auto')
        return translated_text
    except Exception as e:
        return None


In [12]:
def preprocessor(text):
    #Remove HTML tags
    text=BeautifulSoup(text,'html').get_text()
    text = re.sub('<[^>]', '', text)
    emoticons = re.findall('(?::|;|)(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    #letters=re.sub('[^a-zA-Z]',' ',raw_text)
    #words=text.lower().split()
    #remove stopwords
    words=text.lower().split()
    stop_words=set(stopwords.words('english'))
    words=[w for w in words if w not in stop_words]
    return words
    #return text

In [13]:
len(traindata)

159571

In [14]:
len(testdata)

200

In [15]:
len(train)

111700

In [16]:
len(test)

47871

In [17]:
xtrain=[]
for t in train['comment_text']:
    t = ' '.join(preprocessor(t))
    #print(t)
    xtrain.append(t)

/tmp/ipykernel_32/1552596011.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text=BeautifulSoup(text,'html').get_text()
/tmp/ipykernel_32/1552596011.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text=BeautifulSoup(text,'html').get_text()


In [18]:
xtest = []
for t in test['comment_text']:
    xtest.append(' '.join(preprocessor(t)))

  

/tmp/ipykernel_32/1552596011.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text=BeautifulSoup(text,'html').get_text()
/tmp/ipykernel_32/1552596011.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text=BeautifulSoup(text,'html').get_text()


In [19]:
def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in text.split()]

In [20]:
tfidf = TfidfVectorizer(strip_accents=None,
                            lowercase=False,
                            preprocessor=None)

In [21]:
param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__tokenizer': [tokenizer_porter],
                   'clf__penalty': ['l2'],   # 'clf__penalty': ['l1', 'l2'],
                   'clf__C': [10.0]},
                  # {'vect__ngram_range': [(1, 1)],
                  #  'vect__tokenizer': [tokenizer, tokenizer_porter],
                  #  'vect__use_idf': [False],
                  #  'vect__norm': [None],
                  #  'clf__penalty': ['l2'],
                  #  'clf__C': [10.0]},
                  ]

In [22]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain1=train['toxic']
ytrain1=list(ytrain1)
ytest1=test['toxic']
ytest1=list(ytest1)

In [23]:
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf_toxic = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_toxic.fit(xtrain, ytrain1)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [24]:
print(gs_lr_tfidf_toxic.best_score_)
print(gs_lr_tfidf_toxic.best_params_)

0.95911369740376
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [25]:
import sys
sys.setrecursionlimit(100000)
predytest_toxic=gs_lr_tfidf_toxic.predict(xtest)

In [26]:
print(classification_report(ytest1, predytest_toxic,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.87      0.68      0.77      4558
           0       0.97      0.99      0.98     43313

    accuracy                           0.96     47871
   macro avg       0.92      0.84      0.87     47871
weighted avg       0.96      0.96      0.96     47871



In [27]:
ytrain2=train['severe_toxic']
ytrain2=list(ytrain2)
ytest2=test['severe_toxic']
ytest2=list(ytest2)

In [28]:
gs_lr_tfidf_severe_toxic = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_severe_toxic.fit(xtrain, ytrain2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extrac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [29]:
print(gs_lr_tfidf_severe_toxic.best_score_)
print(gs_lr_tfidf_severe_toxic.best_params_)
predytest_severe_toxic=gs_lr_tfidf_severe_toxic.predict(xtest)

0.9903043867502237
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [30]:
print(classification_report(ytest2, predytest_severe_toxic,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.53      0.29      0.37       491
           0       0.99      1.00      1.00     47380

    accuracy                           0.99     47871
   macro avg       0.76      0.64      0.68     47871
weighted avg       0.99      0.99      0.99     47871



In [31]:
ytrain3=train['obscene']
ytrain3=list(ytrain3)
ytest3=test['obscene']
ytest3=list(ytest3)

In [32]:
gs_lr_tfidf_obscene = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_obscene.fit(xtrain, ytrain3)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extrac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [33]:
print(gs_lr_tfidf_obscene.best_score_)
print(gs_lr_tfidf_obscene.best_params_)
predytest_obscene=gs_lr_tfidf_obscene.predict(xtest)

0.9790510295434199
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [34]:
print(classification_report(ytest3, predytest_obscene,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.88      0.70      0.78      2525
           0       0.98      0.99      0.99     45346

    accuracy                           0.98     47871
   macro avg       0.93      0.85      0.88     47871
weighted avg       0.98      0.98      0.98     47871



In [35]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain4=train['threat']
ytrain4=list(ytrain4)
ytest4=test['threat']
ytest4=list(ytest4)

In [36]:
gs_lr_tfidf_threat = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_threat.fit(xtrain, ytrain4)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extrac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [37]:
print(gs_lr_tfidf_threat.best_score_)
print(gs_lr_tfidf_threat.best_params_)
predytest_threat=gs_lr_tfidf_threat.predict(xtest)
#predytestprob_threat=[]

0.9969292748433303
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [38]:
print(classification_report(ytest4,predytest_threat,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.67      0.27      0.38       135
           0       1.00      1.00      1.00     47736

    accuracy                           1.00     47871
   macro avg       0.83      0.63      0.69     47871
weighted avg       1.00      1.00      1.00     47871



In [39]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain5=train['insult']
ytrain5=list(ytrain5)
ytest5=test['insult']
ytest5=list(ytest5)

In [40]:
gs_lr_tfidf_insult = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_insult.fit(xtrain, ytrain5)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extrac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [41]:
print(gs_lr_tfidf_insult.best_score_)
print(gs_lr_tfidf_insult.best_params_)
predytest_insult=gs_lr_tfidf_insult.predict(xtest)
#predytestprob_insult=[]

0.970465532676813
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [42]:
print(classification_report(ytest5,predytest_insult,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.78      0.56      0.65      2355
           0       0.98      0.99      0.98     45516

    accuracy                           0.97     47871
   macro avg       0.88      0.77      0.82     47871
weighted avg       0.97      0.97      0.97     47871



In [43]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain6=train['identity_hate']
ytrain6=list(ytrain6)
ytest6=test['identity_hate']
ytest6=list(ytest6)

In [44]:
gs_lr_tfidf_identity_hate = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_identity_hate.fit(xtrain, ytrain6)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extrac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer_porter at 0x7fce4b27f9a0>]}],
             scoring='accuracy', verbose=1)

In [45]:
print(gs_lr_tfidf_identity_hate.best_score_)
print(gs_lr_tfidf_identity_hate.best_params_)
predytest_identity_hate=gs_lr_tfidf_identity_hate.predict(xtest)
#predytestprob_identity_hate=[]

0.9921665174574754
{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7fce4b27f9a0>}


In [46]:
print(classification_report(ytest6, predytest_identity_hate,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.67      0.25      0.36       449
           0       0.99      1.00      1.00     47422

    accuracy                           0.99     47871
   macro avg       0.83      0.62      0.68     47871
weighted avg       0.99      0.99      0.99     47871



In [47]:
from sklearn.metrics import accuracy_score

In [48]:
#traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
a1=accuracy_score(ytest1,predytest_toxic)
a2=accuracy_score(ytest2,predytest_severe_toxic)
a3=accuracy_score(ytest3,predytest_obscene)
a4=accuracy_score(ytest4,predytest_threat)
a5=accuracy_score(ytest5,predytest_insult)
a6=accuracy_score(ytest6,predytest_identity_hate)
a=(a1+a2+a3+a4+a5+a6)/6
print("average accuracy:",a)

average accuracy: 0.9814675551656187


In [49]:
from sklearn.metrics import hamming_loss

In [50]:
predydataF={'1':predytest_toxic,'2':predytest_severe_toxic,'3':predytest_obscene,'4':predytest_threat,'5':predytest_insult,
            '6':predytest_identity_hate}
predydataF=pd.DataFrame(predydataF)
predydataF=predydataF.values

In [51]:
testy_all=test[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
testy_all=testy_all.values

In [52]:
xtraintf=xtest+xtrain

In [53]:
xtesttf = []
for t in testdata['comment_text']:
    t = ' '.join(preprocessor(t))
    t = translate_to_english(t)
    if(t!=None):
        xtesttf.append(t)
    
    

/tmp/ipykernel_32/1552596011.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text=BeautifulSoup(text,'html').get_text()


In [54]:
probs_ytest = gs_lr_tfidf_toxic.predict_proba(xtesttf)
probs_ytest_severe_toxic = gs_lr_tfidf_severe_toxic.predict_proba(xtesttf)
probs_ytest_obscene = gs_lr_tfidf_obscene.predict_proba(xtesttf)
probs_ytest_threat = gs_lr_tfidf_threat.predict_proba(xtesttf)
probs_ytest_insult = gs_lr_tfidf_insult.predict_proba(xtesttf)
probs_ytest_identity_hate = gs_lr_tfidf_identity_hate.predict_proba(xtesttf)

# Combine the probabilities into a single DataFrame
probs_df = pd.DataFrame({'id': testdata['id'],
                          'toxic': probs_ytest[:, 1],
                          'severe_toxic': probs_ytest_severe_toxic[:, 1],
                          'obscene': probs_ytest_obscene[:, 1],
                          'threat': probs_ytest_threat[:, 1],
                          'insult': probs_ytest_insult[:, 1],
                          'identity_hate': probs_ytest_identity_hate[:, 1]})

# Create the submission file
probs_df.to_csv('submission_probs.csv', index=False)